In [1]:
from sklearn.datasets import load_digits
import random
import numpy as np
import pandas as pd
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import LearningRateScheduler, Callback

/home/pa/.virtualenvs/sandbox/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pa/.virtualenvs/sandbox/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pa/.virtualenvs/sandbox/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pa/.virtualenvs/sandbox/lib/python3.6/site-packages/ten

# Load MNIST

In [2]:
digits = load_digits()

X, y = digits.data[:-1], digits.target[:-1]

In [3]:
input_dim = X.shape[1]
num_classes = np.unique(y).shape[0]
lr = 0.1

y = pd.get_dummies(y).values

# Define simple network and train it

In [47]:
def get_model(input_dim, num_classes, lr):
    
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer=SGD(lr=lr),
                  metrics=['accuracy'])
    
    return model

In [92]:
def scheduler(epoch):
    return loss_history.lr[-1]

In [96]:
class LossHistory(Callback):
    
    def __init__(self, verbose):
        self.verbose = verbose
        self.losses = [np.inf]
        self.acc = [0.0]
        self.lr = [0.1]
 
    def on_epoch_end(self, epoch, logs={}):
        
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('acc'))
        self.lr.append(scheduler(len(self.losses)))
        if epoch % 2 == 0:
            print('learning rate: {}'.format(np.round(self.lr[-1], 4)))
        
#        if epoch % self.verbose == 0:
#            # you can access loss, accuracy in self.params['metrics']
#            print('{} - loss: {} - acc: {}\n'.format(epoch, self.losses[-1], self.acc[-1]))

In [97]:
model = get_model(input_dim, num_classes, lr)

loss_history = LossHistory(verbose=10)
lrate = LearningRateScheduler(scheduler)
callbacks_list = [loss_history, lrate]
history = model.fit(X, y, 
                   epochs=10000, 
                   batch_size=64, 
                   callbacks=callbacks_list, 
                   verbose=0)


learning rate: 0.1
learning rate: 0.1
learning rate: 0.1


KeyboardInterrupt: 

# Define DQN

In [53]:
import gym

In [54]:
class DQNAgent():
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [57]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [65]:
env.reset()

array([ 0.00966972,  0.01622245,  0.0247398 , -0.00710952])

In [67]:
def reset():
    model = get_model(input_dim, num_classes, lr)

    loss_history = LossHistory(verbose=10)
    lrate = LearningRateScheduler(scheduler)
    
    return model, loss_history, lrate

In [80]:
loss_history.losses

[inf]

In [102]:
EPISODES = 100
IMPROVE = 0.0

env = gym.make('CartPole-v1')
state_size = 3
action_size = 2
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    
    state = [np.inf, 0.0, lr]
    state = np.reshape(state, [1, state_size])
    model, loss_history, lrate = reset()
    
    for time in range(100):
        
        action = agent.act(state)
        loss_history.lr.append(loss_history.lr[-1] - (1 - 2 * action) * 0.001)
        callbacks_list = [loss_history, lrate]
        
        model.fit(X, y, 
                   epochs=1, 
                   batch_size=64, 
                   callbacks=callbacks_list, 
                   verbose=2)
        
        next_state = loss_history.losses[-1], loss_history.acc[-1], loss_history.lr[-1]
        
        improvement = loss_history.acc[-1] - loss_history.acc[-2]
        reward = loss_history.acc[-1] + IMPROVE * improvement
        done = improvement == 0.0
        reward = -0.1 if improvement < 0.0 else reward
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

learning rate: 0.099
1796/1796 - 1s - loss: 2.0244 - acc: 0.5423
learning rate: 0.1
1796/1796 - 0s - loss: 3.7270 - acc: 0.4104
learning rate: 0.101
1796/1796 - 0s - loss: 4.2895 - acc: 0.2567
learning rate: 0.1
1796/1796 - 0s - loss: 0.9946 - acc: 0.6531
learning rate: 0.101
1796/1796 - 0s - loss: 0.4437 - acc: 0.8402
learning rate: 0.102
1796/1796 - 0s - loss: 0.6082 - acc: 0.8157
learning rate: 0.103
1796/1796 - 0s - loss: 0.1542 - acc: 0.9504
learning rate: 0.102
1796/1796 - 0s - loss: 0.0911 - acc: 0.9710
learning rate: 0.101
1796/1796 - 0s - loss: 0.0761 - acc: 0.9755
learning rate: 0.102
1796/1796 - 0s - loss: 0.0491 - acc: 0.9827
learning rate: 0.101
1796/1796 - 0s - loss: 0.0349 - acc: 0.9900
learning rate: 0.1
1796/1796 - 0s - loss: 0.2551 - acc: 0.9304
learning rate: 0.101
1796/1796 - 0s - loss: 0.0386 - acc: 0.9900
learning rate: 0.102
1796/1796 - 0s - loss: 0.0233 - acc: 0.9928
learning rate: 0.103
1796/1796 - 0s - loss: 0.0252 - acc: 0.9939
learning rate: 0.102
1796/1796 

learning rate: 0.1
1796/1796 - 0s - loss: 2.4510 - acc: 0.0963
learning rate: 0.101
1796/1796 - 0s - loss: 2.3898 - acc: 0.0958
learning rate: 0.1
1796/1796 - 0s - loss: 2.3526 - acc: 0.0952
learning rate: 0.099
1796/1796 - 0s - loss: 2.3326 - acc: 0.0963
learning rate: 0.1
1796/1796 - 0s - loss: 2.3213 - acc: 0.0952
learning rate: 0.101
1796/1796 - 0s - loss: 2.3138 - acc: 0.1013
learning rate: 0.102
1796/1796 - 0s - loss: 2.3086 - acc: 0.1013
episode: 6/100, score: 65, e: 0.62
learning rate: 0.099
1796/1796 - 1s - loss: 3.2181 - acc: 0.3313
learning rate: 0.1
1796/1796 - 0s - loss: 0.6269 - acc: 0.8018
learning rate: 0.099
1796/1796 - 0s - loss: 3.6071 - acc: 0.4582
learning rate: 0.1
1796/1796 - 0s - loss: 0.5868 - acc: 0.8218
learning rate: 0.099
1796/1796 - 0s - loss: 1.6363 - acc: 0.5451
learning rate: 0.098
1796/1796 - 0s - loss: 0.4548 - acc: 0.8747
learning rate: 0.099
1796/1796 - 0s - loss: 4.1621 - acc: 0.4059
learning rate: 0.1
1796/1796 - 0s - loss: 0.9173 - acc: 0.7450
le

learning rate: 0.1
1796/1796 - 0s - loss: 0.8569 - acc: 0.7366
learning rate: 0.099
1796/1796 - 0s - loss: 1.2656 - acc: 0.6782
learning rate: 0.098
1796/1796 - 0s - loss: 0.2594 - acc: 0.9215
learning rate: 0.099
1796/1796 - 0s - loss: 0.1989 - acc: 0.9393
learning rate: 0.098
1796/1796 - 0s - loss: 0.0794 - acc: 0.9749
learning rate: 0.097
1796/1796 - 0s - loss: 0.0531 - acc: 0.9861
learning rate: 0.096
1796/1796 - 0s - loss: 0.0540 - acc: 0.9844
learning rate: 0.095
1796/1796 - 0s - loss: 0.0236 - acc: 0.9933
learning rate: 0.094
1796/1796 - 0s - loss: 0.0139 - acc: 0.9972
learning rate: 0.093
1796/1796 - 0s - loss: 0.0236 - acc: 0.9944
learning rate: 0.092
1796/1796 - 0s - loss: 0.0103 - acc: 0.9978
learning rate: 0.093
1796/1796 - 0s - loss: 0.0602 - acc: 0.9827
learning rate: 0.092
1796/1796 - 0s - loss: 0.0358 - acc: 0.9911
learning rate: 0.091
1796/1796 - 0s - loss: 0.0086 - acc: 0.9972
learning rate: 0.09
1796/1796 - 0s - loss: 0.0059 - acc: 0.9989
learning rate: 0.089
1796/17

KeyboardInterrupt: 

# Define environement

In [54]:
class DQNAgent():
    
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [56]:
EPISODES = 1000

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
# agent.load("./save/cartpole-dqn.h5")
done = False
batch_size = 32

for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, time, agent.epsilon))
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

episode: 0/1000, score: 16, e: 1.0
episode: 1/1000, score: 11, e: 1.0
episode: 2/1000, score: 16, e: 0.94
episode: 3/1000, score: 10, e: 0.89
episode: 4/1000, score: 55, e: 0.68
episode: 5/1000, score: 10, e: 0.64
episode: 6/1000, score: 19, e: 0.58
episode: 7/1000, score: 16, e: 0.54
episode: 8/1000, score: 11, e: 0.51
episode: 9/1000, score: 12, e: 0.48
episode: 10/1000, score: 10, e: 0.46
episode: 11/1000, score: 12, e: 0.43
episode: 12/1000, score: 9, e: 0.41
episode: 13/1000, score: 10, e: 0.39
episode: 14/1000, score: 8, e: 0.38
episode: 15/1000, score: 10, e: 0.36
episode: 16/1000, score: 8, e: 0.34
episode: 17/1000, score: 9, e: 0.33
episode: 18/1000, score: 8, e: 0.32
episode: 19/1000, score: 144, e: 0.15
episode: 20/1000, score: 125, e: 0.082
episode: 21/1000, score: 57, e: 0.062
episode: 22/1000, score: 43, e: 0.05
episode: 23/1000, score: 17, e: 0.046
episode: 24/1000, score: 50, e: 0.035
episode: 25/1000, score: 23, e: 0.032
episode: 26/1000, score: 20, e: 0.029


KeyboardInterrupt: 